In [2]:
import numpy as np
import pandas as pd
import os
import re
from utils import _strip_string,delete_extra_zero

In [192]:
MODEL_NAME = 'Llama2'
CATEGORY = 'last_letters'
DF_NAME = 'last_letters'
dir_pth = f'../data/ES_data/{MODEL_NAME}_result'
df_t07 = pd.read_json(os.path.join(dir_pth,f'{DF_NAME}/T0.7.jsonl'), lines=True)

In [193]:
# import json
# reformated_dict = {
#     'question':[],
#     'answer': [],
#     'generated_answer': []
# }
# for i in df_t07.columns:
#     data_dict = json.loads(str(df_t07[i].values[0]))
#     for key,val in data_dict.items():
#         reformated_dict[key].append(val)
# df_t07 = pd.DataFrame(reformated_dict)
df_t07

,question,answer,generated_answer
0,"Take the last letters of each words in ""Whitne...",yajo,"[A: The last letter of ""Whitney"" is ""y"". The l..."
1,"Take the last letters of each words in ""Lucky ...",yajc,"[A: The last letter of ""Lucky"" is ""y"". The las..."
2,"Take the last letters of each words in ""Caleb ...",berl,"[A: Sure! The last letters of each word in ""Ca..."
3,"Take the last letters of each words in ""Silvia...",aank,"[A: The last letter of ""Silvia"" is ""a"". The la..."
4,"Take the last letters of each words in ""Breann...",ayre,[A: Sure! Here are the last letters of each wo...
...,...,...,...
495,"Take the last letters of each words in ""Mya Fe...",aoay,[A: Sure! Here are the last letters of each wo...
496,"Take the last letters of each words in ""Emanue...",leze,"[A: The last letter of ""Emanuel"" is ""l"". The l..."
497,"Take the last letters of each words in ""Len Ma...",nsea,"[A: The last letter of ""Len"" is ""n"". The last ..."
498,"Take the last letters of each words in ""Rich P...",hrds,[A: Sure! Here are the last letters of each wo...


In [194]:
len(df_t07['generated_answer'][0])

40

# GSM8K

In [195]:
# ANS_RE = re.compile(r"#### (\-?[0-9\.\,]+)")
# INVALID_ANS = "[invalid]"
# 
# 
# def extract_answer(completion):
#     match = ANS_RE.search(completion)
#     if match:
#         match_str = match.group(1).strip()
#         match_str = match_str.replace(",", "")
#         return match_str
#     else:
#         return INVALID_ANS
# 
# 
# def extract_math_answer(pred_str):
#     if ('The answer is ' in pred_str):
#         pred = pred_str.split('The answer is ')[-1].strip()
#     elif ('the answer is ' in pred_str):
#         pred = pred_str.split('the answer is ')[-1].strip()
#     elif 'boxed' in pred_str:
#         ans = pred_str.split('boxed')[-1]
#         if (ans[0] == '{'):
#             stack = 1
#             a = ''
#             for c in ans[1:]:
#                 if (c == '{'):
#                     stack += 1
#                     a += c
#                 elif (c == '}'):
#                     stack -= 1
#                     if (stack == 0): break
#                     a += c
#                 else:
#                     a += c
#         else:
#             a = ans.split('$')[0].strip()
#         a = _strip_string(a)
#         pred = a
# 
#     else:
#         pattern = '-?\d*\.?\d+'
#         pred = re.findall(pattern, pred_str)
#         if (len(pred) >= 1):
#             # print(pred_str)
#             pred = pred[-1]
#         else:
#             pred = ''
#     if pred != "":
#         if pred[-1] == ".":
#             pred = pred[:-1]
#         if pred[-1] == "/":
#             pred = pred[:-1]
#     pred = _strip_string(pred)
#     if 'boxed' in pred:
#         ans = pred.split('boxed')[-1]
#         if (ans[0] == '{'):
#             stack = 1
#             a = ''
#             for c in ans[1:]:
#                 if (c == '{'):
#                     stack += 1
#                     a += c
#                 elif (c == '}'):
#                     stack -= 1
#                     if (stack == 0): break
#                     a += c
#                 else:
#                     a += c
#         else:
#             a = ans.split('$')[0].strip()
#         a = _strip_string(a)
#         pred = a
#     return pred

In [196]:
def save_csv(row):
    storage_dir = os.path.join('../data', f'ES_data/{MODEL_NAME}_result/{DF_NAME}')
    if not os.path.exists(storage_dir):
        os.makedirs(storage_dir)
    file_path = os.path.join(storage_dir, f'{DF_NAME}_cleaned.csv')

    if not os.path.isfile(file_path):
        # The file does not exist, write with header
        row.to_frame().T.to_csv(file_path, mode='a', index=False, header=True)
    else:
        # The file exists, append without header
        row.to_frame().T.to_csv(file_path, mode='a', index=False, header=False)

In [197]:
# save_dict = {
#     'Category': [CATEGORY for i in range(len(df_t07))],
#     'Question':[None for i in range(len(df_t07))],
#     'Correct Answer': [None for i in range(len(df_t07))],
# }
# df_save = pd.DataFrame(save_dict)
# for row_idx in range(len(df_t07)):
#     row = df_t07.iloc[row_idx]
#     row_save = df_save.iloc[row_idx]
#     row_save['Question']= row['question']
#     right_answer = delete_extra_zero(extract_answer(row['answer']))
#     row_save['Correct Answer'] = right_answer
#     cot_li = row['generated_answer']
#     for idx,cot in enumerate(cot_li):
#         tem_1 = cot
#         number_list = re.findall(r"\d+\.?\d*", tem_1)
#         try:
#             predict1 = number_list[-1].strip('.')
#             row_save[f'CoT_{idx}'] = cot
#             row_save[f'Final Answer_{idx}'] = predict1
#             row_save[f'Instruction Violation_{idx}'] = [(0,0)]
#         except:
#             predict1 = -1000
#             row_save[f'CoT_{idx}'] = 'ERROR'
#             row_save[f'Final Answer_{idx}'] = 'error'
#             row_save[f'Instruction Violation_{idx}'] = [(0, 1), (0, 2), (0, 3), (0, 4)]
#             # continue
# 
#     save_csv(row_save)

# Common and Last Letter

In [198]:
# # common
# def extract_answer(generated_answer):
#     answer_text = generated_answer.split('the answer is')[-1]
#     _ = answer_text
#     p = re.compile(r'[(](.*)[)]', re.S)
#     answer_text = re.findall(p, answer_text)
#     if answer_text:
#         return answer_text[0]
#     else:
#         return _
# last letter
def extract_answer(generated_answer):
    answer_text = generated_answer.lower().split('the answer is')[-1]
    answer_text = ''.join(re.split(r'[^A-Za-z]', answer_text))
    return answer_text

In [199]:
import numpy as np
save_dict = {
    'Category': [CATEGORY for i in range(len(df_t07))],
    'Question':[None for i in range(len(df_t07))],
    'Correct Answer': [None for i in range(len(df_t07))],
}
df_save = pd.DataFrame(save_dict)
for row_idx in range(len(df_t07)):
    row = df_t07.iloc[row_idx]
    row_save = df_save.iloc[row_idx]
    row_save['Question']= row['question']
    right_answer = row['answer'].lower()
    row_save['Correct Answer'] = right_answer
    cot_li = row['generated_answer']
    for idx,cot in enumerate(cot_li):
        tem_1 = cot
        predict1 = extract_answer(tem_1)
        if len(cot)==0:
            row_save[f'CoT_{idx}'] = 'A: Error'
            row_save[f'Final Answer_{idx}'] = 'Error'
            row_save[f'Instruction Violation_{idx}'] = [(0, 1), (0, 2), (0, 3), (0, 4)] 
        else:
            row_save[f'CoT_{idx}'] = cot
            row_save[f'Final Answer_{idx}'] = predict1
            row_save[f'Instruction Violation_{idx}'] = [(0,0)]
    save_csv(row_save)

# Strategy

In [153]:
# def find_answer(sample):
#     if sample['answer']['Yes']:
#         return 1
#     else:
#         return 0
# 
# 
# def extract_answer(generated_answer):
#     if 'the answer is yes' in generated_answer.lower():
#         return 1
#     elif 'the answer is no' in generated_answer.lower():
#         return 0
#     else:
#         return -1

In [155]:
# import numpy as np
# save_dict = {
#     'Category': [CATEGORY for i in range(len(df_t07))],
#     'Question':[None for i in range(len(df_t07))],
#     'Correct Answer': [None for i in range(len(df_t07))],
# }
# df_save = pd.DataFrame(save_dict)
# for row_idx in range(len(df_t07)):
#     row = df_t07.iloc[row_idx]
#     row_save = df_save.iloc[row_idx]
#     row_save['Question']= row['question']
#     right_answer = find_answer(row)
#     row_save['Correct Answer'] = right_answer
#     cot_li = row['generated_answer']
#     for idx,cot in enumerate(cot_li):
#         tem_1 = cot
#         predict1 = extract_answer(tem_1)
#         if len(cot) == 0:
#             cot = 'A: Error'
#         if predict1==-1:
#             row_save[f'CoT_{idx}'] = cot
#             row_save[f'Final Answer_{idx}'] = 'Error'
#             row_save[f'Instruction Violation_{idx}'] = [(0, 1), (0, 2), (0, 3), (0, 4)] 
#         else:
#             row_save[f'CoT_{idx}'] = cot
#             row_save[f'Final Answer_{idx}'] = predict1
#             row_save[f'Instruction Violation_{idx}'] = [(0,0)]
#     save_csv(row_save)